In [3]:
from biogrid.db.models import Interaction, Protein, Organism
from biogrid.db.manager import Importer, Query
import pandas as pd
import sqlalchemy 
import os
from sqlalchemy import Engine, create_engine
connection_string ='sqlite:///biogrid_coronavirus.db'


In [4]:
# This is the original dataset file. If you wish to move the data in the /tests/data folder, use this:
# os.path.join('tests', 'data', 'BIOGRID-CORONAVIRUS-4.4.242.tab3.tsv')
from typing import LiteralString
from pandas import DataFrame


original_corona_dataset_file_path: LiteralString = os.path.join('../BIOGRID-CORONAVIRUS-4.4.242.tab3.tsv')

# This line shows the dataframe. nrows= n shows only the first n rows and also copies these many rows later to a
# separate .tsv file which contains this mini data. You can change the rows to be displayed as you like.
# This is done because the large tsv file crashes the VS code in weaker computers when converting to a .db file
original_dataframe: DataFrame = pd.read_csv(filepath_or_buffer=original_corona_dataset_file_path, sep='\t', nrows=5000)
original_dataframe

,#BioGRID Interaction ID,Entrez Gene Interactor A Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,Synonyms Interactor B,...,TREMBL Accessions Interactor B,REFSEQ Accessions Interactor B,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types,Organism Name Interactor A,Organism Name Interactor B
141211,850980,852501,31542,32899,YLR274W,YBR202W,MCM5,MCM7,BOB1|CDC46|MCM DNA helicase complex subunit MC...,CDC47|MCM DNA helicase complex subunit MCM7|L0...,...,-,NP_009761,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
141212,852501,850980,32899,31542,YBR202W,YLR274W,MCM7,MCM5,CDC47|MCM DNA helicase complex subunit MCM7|L0...,BOB1|CDC46|MCM DNA helicase complex subunit MC...,...,-,NP_013376,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
141213,850980,856680,31542,36697,YLR274W,YEL032W,MCM5,MCM3,BOB1|CDC46|MCM DNA helicase complex subunit MC...,MCM DNA helicase complex subunit MCM3|L000001039,...,-,NP_010882,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
141214,856680,850980,36697,31542,YEL032W,YLR274W,MCM3,MCM5,MCM DNA helicase complex subunit MCM3|L000001039,BOB1|CDC46|MCM DNA helicase complex subunit MC...,...,-,NP_013376,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
141215,852501,850793,32899,31376,YBR202W,YLR103C,MCM7,CDC45,CDC47|MCM DNA helicase complex subunit MCM7|L0...,SLD4|L000003380,...,-,NP_013204,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2792037,43740578,4677,4383851,110758,GU280_gp01_nsp2,-,nsp2,NARS,ORF1ab|ORF1ab-nsp2|SARS-CoV2 nsp2|SARS-CoV-2 n...,ASNRS|NARS1,...,-,NP_004530,-,-,-,-,-,-,Severe acute respiratory syndrome coronavirus 2,Homo sapiens
2792038,43740578,488,4383851,106978,GU280_gp01_nsp2,-,nsp2,ATP2A2,ORF1ab|ORF1ab-nsp2|SARS-CoV2 nsp2|SARS-CoV-2 n...,ATP2B|DAR|DD|SERCA2,...,-,NP_001672|NP_733765,-,-,-,-,-,-,Severe acute respiratory syndrome coronavirus 2,Homo sapiens
2792039,43740578,5501,4383851,111495,GU280_gp01_nsp2,-,nsp2,PPP1CC,ORF1ab|ORF1ab-nsp2|SARS-CoV2 nsp2|SARS-CoV-2 n...,PP-1G|PP1C|PPP1G,...,A0A024RBP2,NP_001231903|NP_002701,-,-,-,-,-,-,Severe acute respiratory syndrome coronavirus 2,Homo sapiens
2792040,43740578,5910,4383851,111845,GU280_gp01_nsp2,-,nsp2,RAP1GDS1,ORF1ab|ORF1ab-nsp2|SARS-CoV2 nsp2|SARS-CoV-2 n...,GDS1|SmgGDS,...,B7Z4M5|B3KNU0|Q6U7G8,NP_066982|NP_001093900|NP_001093899|NP_0010938...,-,-,-,-,-,-,Severe acute respiratory syndrome coronavirus 2,Homo sapiens


In [5]:
# Define the file path
from typing import LiteralString
import csv
corona_file_path: LiteralString = os.path.join('../tests', 'data', 'BIOGRID-CORONAVIRUS-test.tsv')

In [6]:
# Save the DataFrame to a TSV file, overwriting if it already exists
# Without the quoting argument, the function .to_csv() encloses all the words in quotes which creates problems
# # escapechar = '\\' needs to be added when quoting= is used 
original_dataframe.to_csv(path_or_buf=corona_file_path, sep='\t', index=False, quoting=csv.QUOTE_NONE, escapechar='\\') 

In [7]:
from typing import LiteralString
df: pd.DataFrame = pd.read_csv(filepath_or_buffer=corona_file_path, sep='\t')
df

,#BioGRID Interaction ID,Entrez Gene Interactor A Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,Synonyms Interactor B,...,TREMBL Accessions Interactor B,REFSEQ Accessions Interactor B,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types,Organism Name Interactor A,Organism Name Interactor B
0,850980,852501,31542,32899,YLR274W,YBR202W,MCM5,MCM7,BOB1|CDC46|MCM DNA helicase complex subunit MC...,CDC47|MCM DNA helicase complex subunit MCM7|L0...,...,-,NP_009761,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
1,852501,850980,32899,31542,YBR202W,YLR274W,MCM7,MCM5,CDC47|MCM DNA helicase complex subunit MCM7|L0...,BOB1|CDC46|MCM DNA helicase complex subunit MC...,...,-,NP_013376,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
2,850980,856680,31542,36697,YLR274W,YEL032W,MCM5,MCM3,BOB1|CDC46|MCM DNA helicase complex subunit MC...,MCM DNA helicase complex subunit MCM3|L000001039,...,-,NP_010882,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
3,856680,850980,36697,31542,YEL032W,YLR274W,MCM3,MCM5,MCM DNA helicase complex subunit MCM3|L000001039,BOB1|CDC46|MCM DNA helicase complex subunit MC...,...,-,NP_013376,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
4,852501,850793,32899,31376,YBR202W,YLR103C,MCM7,CDC45,CDC47|MCM DNA helicase complex subunit MCM7|L0...,SLD4|L000003380,...,-,NP_013204,-,-,-,-,-,-,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,43740578,4677,4383851,110758,GU280_gp01_nsp2,-,nsp2,NARS,ORF1ab|ORF1ab-nsp2|SARS-CoV2 nsp2|SARS-CoV-2 n...,ASNRS|NARS1,...,-,NP_004530,-,-,-,-,-,-,Severe acute respiratory syndrome coronavirus 2,Homo sapiens
4996,43740578,488,4383851,106978,GU280_gp01_nsp2,-,nsp2,ATP2A2,ORF1ab|ORF1ab-nsp2|SARS-CoV2 nsp2|SARS-CoV-2 n...,ATP2B|DAR|DD|SERCA2,...,-,NP_001672|NP_733765,-,-,-,-,-,-,Severe acute respiratory syndrome coronavirus 2,Homo sapiens
4997,43740578,5501,4383851,111495,GU280_gp01_nsp2,-,nsp2,PPP1CC,ORF1ab|ORF1ab-nsp2|SARS-CoV2 nsp2|SARS-CoV-2 n...,PP-1G|PP1C|PPP1G,...,A0A024RBP2,NP_001231903|NP_002701,-,-,-,-,-,-,Severe acute respiratory syndrome coronavirus 2,Homo sapiens
4998,43740578,5910,4383851,111845,GU280_gp01_nsp2,-,nsp2,RAP1GDS1,ORF1ab|ORF1ab-nsp2|SARS-CoV2 nsp2|SARS-CoV-2 n...,GDS1|SmgGDS,...,B7Z4M5|B3KNU0|Q6U7G8,NP_066982|NP_001093900|NP_001093899|NP_0010938...,-,-,-,-,-,-,Severe acute respiratory syndrome coronavirus 2,Homo sapiens


In [8]:
engine: Engine = create_engine(connection_string)
# The path for the .tsv file containing the small dataset is used here
importer = Importer(engine=engine, file_path=corona_file_path) 
loaded_data = importer.load_data()
loaded_data


,biogrid_interaction_id,official_symbol_interactor_a,official_symbol_interactor_b,experimental_system,experimental_system_type,organism_id_interactor_a,organism_id_interactor_b,score,swiss_prot_accessions_interactor_a,swiss_prot_accessions_interactor_b,organism_name_interactor_a,organism_name_interactor_b
0,850980,MCM5,MCM7,Two-hybrid,physical,559292,559292,-,P29496,P38132,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
1,852501,MCM7,MCM5,Two-hybrid,physical,559292,559292,-,P38132,P29496,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
2,850980,MCM5,MCM3,Two-hybrid,physical,559292,559292,-,P29496,P24279,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
3,856680,MCM3,MCM5,Two-hybrid,physical,559292,559292,-,P24279,P29496,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
4,852501,MCM7,CDC45,Two-hybrid,physical,559292,559292,-,P38132,Q08032,Saccharomyces cerevisiae (S288c),Saccharomyces cerevisiae (S288c)
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,43740578,nsp2,NARS,Proximity Label-MS,physical,2697049,9606,2.5,P0DTD1,O43776,Severe acute respiratory syndrome coronavirus 2,Homo sapiens
4996,43740578,nsp2,ATP2A2,Proximity Label-MS,physical,2697049,9606,3.55,P0DTD1,P16615,Severe acute respiratory syndrome coronavirus 2,Homo sapiens
4997,43740578,nsp2,PPP1CC,Proximity Label-MS,physical,2697049,9606,6.1,P0DTD1,P36873,Severe acute respiratory syndrome coronavirus 2,Homo sapiens
4998,43740578,nsp2,RAP1GDS1,Proximity Label-MS,physical,2697049,9606,7.32,P0DTD1,P52306,Severe acute respiratory syndrome coronavirus 2,Homo sapiens


In [9]:
# This is just for debugging purposes to find any primary keys which are repeated. If repeated then it gives an error when importing the data using .import_data():
# An error occurred: (sqlite3.IntegrityError) UNIQUE constraint failed: protein.uniprot_id

df_proteins = importer.get_proteins_df(df)
duplicates = df_proteins[df_proteins.duplicated(subset=['uniprot_id'], keep=False)]
print("Duplicate Proteins Found:\n", duplicates)


Duplicate Proteins Found:
 Empty DataFrame
Columns: [uniprot_id, symbol, tax_id]
Index: []


In [10]:
# This is just for debugging purposes to find any primary keys which are repeated.
df_organisms = importer.get_organisms_df(df)
duplicates = df_organisms[df_organisms.duplicated(subset=['tax_id'], keep=False)]
print("Duplicate Organisms Found:\n", duplicates)


Duplicate Organisms Found:
 Empty DataFrame
Columns: [tax_id, name]
Index: []


In [11]:
# This is just for debugging purposes to find any primary keys which are repeated.
df_organisms = importer.get_interaction_df(df)
duplicates = df_organisms[df_organisms.duplicated(subset=['id'], keep=False)]
print("Duplicate Interaction Found:\n", duplicates)

Duplicate Interaction Found:
 Empty DataFrame
Columns: [id, interactor_a_id, interactor_b_id, score, experimental_system, experimental_system_type]
Index: []


In [12]:
importer.import_data()

In [13]:
importer.get_interaction_df(loaded_data)

,id,interactor_a_id,interactor_b_id,score,experimental_system,experimental_system_type
0,850980,P29496,P38132,None,Two-hybrid,physical
1,852501,P38132,P29496,None,Two-hybrid,physical
2,856680,P24279,P29496,None,Two-hybrid,physical
3,854391,Q12234,P40395,None,Synthetic Lethality,genetic
4,595,P24385,P03372,None,Affinity Capture-Western,physical
...,...,...,...,...,...,...
99,79412,Q8NCW0,P0DTC2,None,Reconstituted Complex,physical
100,10288,None,P0DTC2,None,Reconstituted Complex,physical
101,10960,Q12907,P0DTC2,None,Reconstituted Complex,physical
102,4247,Q10469,P0DTC2,None,Reconstituted Complex,physical


In [14]:
importer.get_proteins_df(loaded_data)

,uniprot_id,symbol,tax_id
0,P29496,MCM5,559292
1,P38132,MCM7,559292
2,P24279,MCM3,559292
3,Q12234,RUD3,559292
4,P24385,CCND1,9606
...,...,...,...
2446,Q13136,PPFIA1,9606
2447,O00170,AIP,9606
2448,P31749,AKT1,9606
2449,Q9UMY4,SNX12,9606


In [15]:
importer.get_organisms_df(loaded_data)

,tax_id,name
0,559292,Saccharomyces cerevisiae (S288c)
1,9606,Homo sapiens
2,2697049,Severe acute respiratory syndrome coronavirus 2
3,694009,Severe acute respiratory syndrome-related coro...
4,10029,Cricetulus griseus
5,1335626,Middle-East Respiratory Syndrome-related Coron...
6,9986,Oryctolagus cuniculus
7,9685,Felis Catus
8,9615,Canis familiaris
9,9544,Macaca mulatta


In [ ]:
query = Query(engine=engine)
query.count_interactions()

Number of interactions: 
102


In [17]:
query.count_proteins()

Number of proteins: 


2450

In [18]:
query.count_proteins()

Number of proteins: 


2450